In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619456 sha256=2840fcf27f4bb5ec07b9993e8401d83083070c4b0adcbb42e3d4a231e4e24d20
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [88]:
import pandas as pd
from surprise import accuracy

In [82]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_dir = '/content/drive/MyDrive/256-project/deezer_features.csv'

In [ ]:
df = pd.read_csv(file_dir, header=None)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,1,-0.007510,0.002892,0.006003,-0.008288,0.001388,-0.009587,-0.002597,-0.001651,0.007500,0.005377,-0.014132,0.024988,0.000546,-0.010660,-0.017597,0.006302,0.023571,-0.013865,0.005722,-0.030666,-0.003915,0.054639,-0.036781,0.018083,-0.004836,-0.001371,-0.018559,0.026180,0.011409,-0.026717,0.009286,-0.029503,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.459391,0.193252,0.139495,0.151400
1,2,-0.172181,-0.082732,0.135843,0.011548,0.017318,-0.254997,0.113355,0.033932,-0.071162,-0.099428,-0.107563,0.042243,0.106874,-0.127894,-0.018735,0.017060,0.076911,-0.127784,-0.183286,-0.182893,0.120741,-0.158713,-0.077712,0.008842,-0.133573,0.030468,-0.225340,0.060964,0.047286,0.195375,0.066941,-0.076980,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.333660,0.161552,0.120748,0.167415
2,3,-0.157183,0.009618,0.088339,-0.084143,0.094306,-0.351647,-0.034583,0.144340,-0.008939,-0.219161,-0.052356,0.286848,0.198618,-0.214440,-0.268811,0.150304,0.448504,-0.384054,0.057444,-0.138773,0.220159,0.288947,-0.352348,-0.050004,0.051974,0.038191,-0.202896,0.417181,0.010661,0.114940,0.244948,0.222384,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.153438,0.186358,0.148314,0.113224
3,4,-0.124824,-0.020829,0.109950,0.005343,-0.014215,-0.075313,-0.015466,0.142181,0.093969,0.049267,-0.040896,-0.058768,0.015671,0.000444,0.055295,-0.003689,0.235109,-0.218155,0.120014,-0.146563,0.243076,-0.017480,-0.035989,0.017321,-0.189579,0.234798,-0.151922,0.161062,-0.054403,0.262250,0.217358,0.176144,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.606095,0.376574,0.045419,0.033385
4,5,-0.261996,-0.288156,0.197021,-0.208817,1.027182,0.207817,-0.073727,0.010425,-0.029520,0.173434,0.076610,-0.029162,-0.006886,0.002491,0.105421,0.066087,-0.081108,0.155968,0.103546,-0.086449,-0.114413,-0.202190,-0.498856,-0.183288,0.200243,0.339281,0.049975,-0.064397,-0.205344,-0.324963,0.335434,-0.181493,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.206979,0.045804,0.141537,0.142070


In [ ]:
df_melted = pd.melt(df, id_vars=[0], value_vars=list(range(1,57)))

In [ ]:
df_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359120 entries, 0 to 1359119
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   0         1359120 non-null  int64  
 1   variable  1359120 non-null  int64  
 2   value     1359120 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 31.1 MB


In [ ]:
df_melted.head(10)

,0,variable,value
0,1,1,-0.007510
1,2,1,-0.172181
2,3,1,-0.157183
3,4,1,-0.124824
4,5,1,-0.261996
5,6,1,-0.010896
6,7,1,-1.713777
7,8,1,-0.007510
8,9,1,-0.890512
9,10,1,-0.004124


In [ ]:
df_melted = df_melted.sort_values(by=['variable'])

In [ ]:
df_country = df_melted[(df_melted.variable > 32) & (df_melted.variable < 53)]

In [ ]:
df_country

,0,variable,value
792817,16178,33,0.0
792818,16179,33,0.0
792823,16184,33,0.0
792820,16181,33,0.0
792821,16182,33,0.0
...,...,...,...
1245859,8090,52,0.0
1245858,8089,52,0.0
1245857,8088,52,0.0
1245856,8087,52,1.0


In [ ]:
df_country = df_country.sort_values(by=0)

In [ ]:
df_c = df_country[df_country[0] <= 100]

In [ ]:
df_c

,0,variable,value
1164960,1,49,0.0
1237770,1,52,0.0
1213500,1,51,0.0
873720,1,37,0.0
922260,1,39,0.0
...,...,...,...
801009,100,34,0.0
922359,100,39,0.0
1116519,100,47,0.0
1043709,100,44,0.0


In [ ]:
train_ds = Dataset.load_from_df(df_c,Reader(rating_scale=(0,1)))

In [ ]:
svd = SVD(n_factors=10, reg_all=0.1, biased=True)

In [ ]:
cross_validate(svd, train_ds, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2332  0.2187  0.1937  0.1841  0.2142  0.2088  0.0177  
MAE (testset)     0.1025  0.0885  0.0847  0.0749  0.0918  0.0885  0.0090  
Fit time          0.03    0.02    0.03    0.03    0.03    0.03    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.02787923812866211,
  0.024306297302246094,
  0.02651357650756836,
  0.025467872619628906,
  0.030232667922973633),
 'test_mae': array([0.10254027, 0.08854902, 0.08473432, 0.07488596, 0.09175325]),
 'test_rmse': array([0.23319248, 0.21868248, 0.19371121, 0.18412423, 0.21422197]),
 'test_time': (0.0030994415283203125,
  0.0027501583099365234,
  0.0028362274169921875,
  0.004204273223876953,
  0.002920866012573242)}

In [80]:
sim_options = {
    'name': 'cosine',
    'user_based':True  
}

In [81]:
knn_bas = surprise.prediction_algorithms.KNNBasic(sim_options=sim_options)

In [84]:
training, testing = train_test_split(train_ds, test_size=0.2, random_state=1, shuffle=True)

In [86]:
knn_bas.fit(training)

Computing the cosine similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.7/dist-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


In [87]:
predictions_knn = knn_bas.test(testing)

In [89]:
accuracy.rmse(predictions_knn, verbose=True) 

RMSE: 0.2138


0.21380481753225297